<a href="https://colab.research.google.com/github/ssook1222/bigdata-analysist-certificate/blob/master/bigdata-analysist-certificate/actual_test/2nd_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 작업 1유형

In [133]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p1_1.csv')
df.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,NaN,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.9,4.98,24.0
1,0.02731,NaN,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.9,9.14,21.6


### 1-1

주어진 Dataset에서 CRIM값이 가장 큰 10개의 지역을 구하고
10개의 지역의 CRIM값을 그 중 가장 작은 값으로 대체하라.        
그리고
AGE 컬럼 값이 80이상인 대체 된 CRIM 평균값을 구하라

! 다시 한 번 풀어보기

In [134]:
df = df.sort_values('CRIM',ascending=False)

In [135]:
df.loc[:9,'CRIM'] = df.loc[:9,'CRIM'].min()

In [136]:
print(df[(df.AGE >= 80)].CRIM.mean())

0.16225033333333336


### 1-2

1-1에서 사용한 데이터에서 RM 중앙값으로 해당 컬럼의 결측치를 대체하라
그리고 해당 컬럼의 결측치 대치 전후의 표준편차 차이의 절댓값을 소숫점 이하 3째자리 까지 구하라

In [137]:
before = df.RM.std()

In [138]:
print(before)

0.7212529033559509


In [139]:
mead = df.median().RM

In [140]:
print(mead)

6.214


In [141]:
df.RM.isna().sum()

38

In [142]:
import numpy as np

In [143]:
def chg_na(x):
  if np.isnan(x):
    return mead
  return x

In [144]:
df['RM'] = df.RM.apply(chg_na)

In [145]:
after = df.RM.std()
print(after)

0.6938906198881796


In [146]:
print(round(abs(after - before), 3))

0.027


### 1-3

주어진 Dataset의 DIS 평균으로 부터 1.5 * 표준편차를 벗어나는 영역을 이상치라고 판단하고
DIS 컬럼의 이상치들의 합을 구하여라.

In [152]:
min_dist = df.mean().DIS - 1.5*df.std().DIS
max_dist = df.mean().DIS + 1.5*df.std().DIS

print(df[((df.DIS < min_dist) | (df.DIS > max_dist))].DIS.sum())

404.4101


---

## 작업 2유형

In [153]:
import pandas as pd
import numpy as np

In [154]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [155]:
from sklearn.tree import DecisionTreeClassifier

In [156]:
from sklearn.metrics import accuracy_score

In [157]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv")

In [161]:
print(x_train.isna().sum())
print(x_train.info())

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gm

In [163]:
x_train.columns

Index(['ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms'],
      dtype='object')

In [164]:
x_train = x_train[['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms']]

In [165]:
x_test = x_test[['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms']]

In [166]:
x_columns = ['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating',  'Product_importance', 'Gender']

In [167]:
le = LabelEncoder()

for column in x_columns:
  x_train[column] = le.fit_transform(x_train[column])
  x_test[column] = le.fit_transform(x_test[column])

In [168]:
dtc = DecisionTreeClassifier()
model = dtc.fit(x_train, y_train)
predict1 = model.predict(x_test)

In [171]:
print(accuracy_score(predict1[:,1], y_test['Reached.on.Time_Y.N']))

0.6398545785048853


In [172]:
submit = pd.DataFrame(data = predict1, columns = y_test.columns)

In [173]:
submit.head(5)

,ID,Reached.on.Time_Y.N
0,6228,1
1,10956,0
2,7008,1
3,5645,1
4,3020,1


In [174]:
submit.to_csv('result.csv')

---

## 작업형 3영역

In [185]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e2_p3_1.csv')
df.head(5)

,fish,height
0,fish_1,33.74cm
1,fish_2,31.64cm
2,fish_3,27.49cm
3,fish_4,31.43cm
4,fish_5,27.56cm


### 3-1-a

122마리의 height 평균값을 m(미터) 단위로 소숫점 이하 5자리까지 실수 값만 출력하라

In [186]:
def chk(x):
  x = x.replace('cm', '')
  x = float(x)
  x = x * 0.01
  return x

In [191]:
def chk2(x):
  x = x.replace('cm', '')
  x = float(x)
  return x

In [187]:
df['height2'] = df.height.apply(chk)

In [192]:
df['height_raw'] = df.height.apply(chk2)

In [188]:
print(round(df.height2.mean(), 5))

0.29951


### 3-1-b

모집단의 평균 길이가 30cm 인지 확인하려 일표본 t 검정을 시행하여 확인하려한다. 검정통계량을 소숫점 이하 3째자리까지 구하여라

! 다시 풀어보기

In [193]:
from scipy.stats import ttest_1samp

s ,p = ttest_1samp(df['height_raw'],30)
result = round(s,3)
print(result)

-0.217


### 3-1-c

위의 통계량에 대한 p-값을 구하고 (반올림하여 소숫점 이하 3째자리), 유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)

In [194]:
print(round(p, 3))
if p < 0.05:
  print('대립가설')
else:
  print('귀무가설')

0.829
귀무가설


---

조사결과 70%의 성인 남성이 3년 동안에 적어도 1번 치과를 찾는다고 할때, 21명의 성인 남성이 임의로 추출되었다고 하자.

### 3-2-a
21명 중 16명 미만이 치과를 찾았을 확률(반올림하여 소숫점 이하 3자리)


! 다시 정리 후 풀어보기

In [197]:
from scipy.stats import binom

In [206]:
import numpy as np

n = 21
p_hat = 0.7
k = 16

In [207]:
# P(X < k) 계산
prob = binom.cdf(k-1, n, p)
print(round(prob, 3))

0.136


### 3-2 - b

적어도 19명이 치과를 찾았을 확률(반올림하여 소숫점 이하 3자리)

In [208]:
import numpy as np

n = 21
p_hat = 0.7
k = 19

In [209]:
# P(X < k) 계산
prob = 1 - binom.cdf(k-1, n, p)
print(round(prob, 3))

0.278
